# https://cmp24.by/c/supermarket/

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import warnings
warnings.simplefilter("ignore")
import time
from itertools import groupby
import os
import random
import json
import datetime

def get_links(url):
    response=requests.get(url)
    soup=BeautifulSoup(response.text,'lxml')
    links=soup.find_all('div',class_='product')
    
    all_main_links=[]
    for i in links:
        try:
            all_main_links.append({                
                'Категория': i.find('div',class_='product_description').text.strip(),
                'Ссылка':'https://cmp24.by'+str(i.find('a',class_='product__link').get('href'))})
        except Exception as _ex:
            break
            
    all_main_links2=[]
    for i in all_main_links:
        response=requests.get(i.get('Ссылка'))
        soup=BeautifulSoup(response.text,'lxml')
        links2=soup.find_all('div',class_='product')
        count=0
        for j in links2:
            try:
                all_main_links2.append({
                    'category':i.get('Категория'),
                    'sub_category': j.find('div',class_='product_description').text.strip(),
                    'Ссылка':'https://cmp24.by'+str(j.find('a',class_='product__link').get('href'))})
            except Exception as _ex:
                break
        if count%5==0:
            time.sleep(2)
            count+=1
        
    
    all_main_links3=[]
    count=0
    for k in all_main_links2:
                   
        for page in range(1,2):    
            response=requests.get('https://cmp24.by/c/supermarket/makaronyi-krupyi-spetsii/spetsii-i-pripravyi/'+str('?p='+str(page)))
            soup=BeautifulSoup(response.text,'lxml')            
            a=soup.find_all('a',{'title':'следующая'})
            if len(a)==1:
                links3=soup.find_all('div',class_='product')
                for l in links3:
                    all_main_links3.append({
                                            'category':k.get('category'),
                                            'sub_category':k.get('sub_category'), 
                                            'Ссылка':'https://cmp24.by'+str(l.find('a').get('href'))})
            else:
                break
            if count%5==0:
                time.sleep(2)
                count+=1
    count=0                         
    all_data=[]
    for i in all_main_links3:
        response=requests.get(i.get('Ссылка'))
        soup=soup=BeautifulSoup(response.text,'lxml')
        title=[]
        brand=[]
        barcode=[]
        try:
            title.append(soup.find('h1',{'itemprop':'name'}).text.strip())
        except Exception as _ex:
            title.append(None)

        try:
            brand.append(soup.find('ul',class_='manufacturer__list').find('span').text.strip())
        except Exception as _ex:
            brand.append(None)    
        try:
            barcode.append(soup.find('ul',class_='manufacturer__list').find('span',{'itemprop':'gtin'}).text.strip())
        except Exception as _ex:
            barcode.append(None)
            
        all_data.append({            
            
            
            'name':title[0],
            'brand':brand[0],
            'barcode':barcode[0], 
            'category':i.get('category'),
            'sub_category':i.get('sub_category')
                                            
                               
                               
                               
                               })
    
        if count%5==0:
            time.sleep(2)
            count+=1
        print(f'processed{count}"/"{len(all_main_links3)}')
            
            
    with open(r'C:\Users\svnduw\Desktop\Bekbol\Bekbol project\barcode\cmp24.bycsupermarket\url.json','w',encoding='utf-8') as file:
                json.dump(all_data,file,indent=4,ensure_ascii=False)
                
            
        
    
            
            

def main():
    get_links('https://cmp24.by/c/supermarket/')
    
    
if __name__=="__main__":
     main()